In [7]:
import requests
import datetime
import time
import pytz
import csv
import warnings
from dateutil import parser
from urllib3.exceptions import InsecureRequestWarning

# Disable urllib3 InsecureRequestWarning
warnings.filterwarnings('ignore', category=InsecureRequestWarning)

from datetime import date, timedelta

class APIReader:


    def __init__(self):
        APIReader.self = self


    def return_daylight():

        # Set timezone to Central European Time
        serbia_tz = pytz.timezone("Europe/Belgrade")

        # Get current time in Serbia
        current_time = datetime.datetime.now(serbia_tz)

        # Determine whether daily saving time is in effect in order to set adjustment
        daylight_saving = current_time.dst() != timedelta(0)
        return daylight_saving


    def collect_data(day):
        combined_data = []

        # Get bounds of data interval which consist of day prior (and day ahead) of today
        yesterday = ""
        tomorrow = ""

        if int(str(day)[8:10]) > 10:
            yesterday = str(day)[0:8] + str(int(str(day)[8:10]) - 1)
        else:
            yesterday = str(day)[0:8] + "0" + str(int(str(day)[8:10]) - 1)

        if int(yesterday[8:10]) > 10: 
            tomorrow = yesterday[0:8] + str(int(yesterday[8:]) + 2)
        else:
            tomorrow = yesterday[0:8] + "0" + str(int(yesterday[8:]) + 2)

        # Note: Zero is added before the day number if day value < 10

        # Get the location of API using bounds around current date

        site_urlcon = "https://ems.energyflux.rs/"
        site_urlprod = "https://ems.energyflux.rs/"
        # https://ems.energyflux.rs/api/consumption/SRBIJA?fromDate=2024-07-22T22:00:00.000Z&toDate=2024-07-24T21:59:59.999Z
        # https://ems.energyflux.rs/api/production/SRBIJA?sourceType=NONE&fromDate=2024-07-22T22:00:00.000Z&toDate=2024-07-24T21:59:59.999Z
        api_urlcon = site_urlcon + "api/consumption/SRBIJA?fromDate=" + yesterday + "T22:00:00.000Z&toDate=" + tomorrow + "T21:59:59.999Z"
        api_urlprod = site_urlprod + "api/production/SRBIJA?sourceType=NONE&fromDate=" + yesterday + "T22:00:00.000Z&toDate=" + tomorrow + "T21:59:59.999Z"

        # New variable set to response from url request
        response_c = requests.get(api_urlcon, verify=False)
        data_c = response_c.json()
        response_p = requests.get(api_urlprod, verify=False)
        data_p = response_p.json()
    
        energy_p = {}
        if response_p.status_code == 200:
            if "production" in data_p:
                production_data = data_p["production"]
            
                for source, values in production_data.items():
                    energy_p[source] = {}
                
                    for ten_min in values:
                        time_key = ten_min["t"]
                        energy_p[source][time_key] = ten_min["v"]
        else:
            print("No Production Data Available for ", str(day))

        energy_c = {}
        if response_c.status_code == 200:
            if "consumption" in data_c:
                consumption_data = data_c["consumption"]
            
                for ten_min in consumption_data:
                    time_key = ten_min["t"]
                    energy_c[time_key] = ten_min["v"]
        else:
            print("No Consumption Data Available for ", str(day))

        combined_data = [energy_c, energy_p]
        return combined_data

    def print_data():
        # Print today's data for prices within Serbian exchange
        today = str(datetime.datetime.today()).split()[0]
        combined_data = APIReader.collect_data(today)

        print("\n")
        print("Serbian Production/Consumption: \n")
        
        energy_c, energy_p = combined_data
        
        # Get all unique timepoints
        timepoints = set(energy_c.keys())
        for source in energy_p.values():
            timepoints.update(source.keys())
        
        for timepoint in sorted(timepoints):
            consumption = energy_c.get(timepoint, "N/A")
            
            production = {}
            for source, values in energy_p.items():
                production[source] = values.get(timepoint, "N/A")
            
            # Parse the timepoint string to a datetime object
            try:
                local_time = parser.parse(timepoint)
                # If the parsed time is naive (no timezone info), assume it's in UTC
                if local_time.tzinfo is None:
                    local_time = pytz.utc.localize(local_time)
                # Convert to Belgrade time
                local_time = local_time.astimezone(pytz.timezone("Europe/Belgrade"))
            except ValueError:
                print(f"Unable to parse time: {timepoint}")
                continue
            
            print(f"Date: {local_time.strftime('%Y-%m-%d')}  Time: {local_time.strftime('%H:%M')}")
            print(f"  Consumption: {consumption}")
            for source, value in production.items():
                print(f"  Production ({source}): {value}")
            print()

if __name__ == "__main__":
    APIReader.print_data()

    





Serbian Production/Consumption: 

Date: 2024-07-25  Time: 00:00
  Consumption: 3403.0
  Production (VETRO): 317.0
  Production (PUMPA): -282.0
  Production (GAS): 178.0
  Production (HIDRO): 834.0
  Production (TERMO): 2294.0

Date: 2024-07-25  Time: 00:10
  Consumption: 3401.0
  Production (VETRO): 319.0
  Production (PUMPA): -282.0
  Production (GAS): 181.0
  Production (HIDRO): 806.0
  Production (TERMO): 2280.0

Date: 2024-07-25  Time: 00:20
  Consumption: 3295.0
  Production (VETRO): 327.0
  Production (PUMPA): -282.0
  Production (GAS): 183.0
  Production (HIDRO): 862.0
  Production (TERMO): 2106.0

Date: 2024-07-25  Time: 00:30
  Consumption: 3301.0
  Production (VETRO): 326.0
  Production (PUMPA): -282.0
  Production (GAS): 183.0
  Production (HIDRO): 909.0
  Production (TERMO): 2130.0

Date: 2024-07-25  Time: 00:40
  Consumption: 3212.0
  Production (VETRO): 318.0
  Production (PUMPA): -283.0
  Production (GAS): 182.0
  Production (HIDRO): 798.0
  Production (TERMO): 2115.0
